https://cseweb.ucsd.edu/~jmcauley/datasets.html#amazon_reviews

https://amazon-reviews-2023.github.io/

# Citation
Bridging Language and Items for Retrieval and Recommendation
Yupeng Hou, Jiacheng Li, Zhankui He, An Yan, Xiusi Chen, Julian McAuley
arXiv
[pdf](https://urldefense.com/v3/__https://arxiv.org/pdf/2403.03952.pdf__;!!Mih3wA!EGkx27nmvdVMhh2uxQ7Mc0rNrXQwV8GsOpd3uSc6ZjJGAVhgy9o5bn3Jeb73P4Lz7oL2dIDMdZR4IHI$)

https://scikit-learn.org/0.15/modules/scaling_strategies.html

In [7]:
!pip install pymongo
!pip install pandas
!pip install bs4

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from MongoDB import MongoDB


amazon_reviews_db = MongoDB("Amazon_Reviews")
amazon_items_db = MongoDB("Amazon_Items")
response = requests.get("https://amazon-reviews-2023.github.io/")
soup  = BeautifulSoup(response.text, "html.parser")
file_links = soup.find_all(
    name = "a",
    attrs = {
        "href" : re.compile(r'.*https://mcauleylab.ucsd.edu/public_datasets/data/amazon_2023/raw/.*')
    }
)
stop = False
for file_link in file_links:
    categories = file_link.attrs["href"].split("/")[-1].split(".jsonl.gz")[0].partition("meta_")
    mongo_db = amazon_reviews_db if categories[-1] == "" else amazon_items_db
    category = categories[0] if categories[-1] == "" else categories[-1]
    if category not in mongo_db and stop == False:
        print(category, mongo_db)
        mongo_db.append(
            **{
                "collection_name" : category,
                "df" : pd.read_json(
                    path_or_buf = file_link.attrs["href"], 
                    compression = "gzip",
                    lines = True
                )
            } | (
                {
                    "timeField" : "timestamp"
                } if mongo_db == amazon_reviews_db else {}
            )
        )
        stop = True

Amazon_Fashion Amazon_Reviews


KeyboardInterrupt: 

In [ ]:
from MongoDB import MongoDB
import pandas as pd


amazon_reviews_db = MongoDB("Amazon_Reviews")
amazon_items_db = MongoDB("Amazon_Items")


reviews = pd.concat(
    objs = [
        batch
        for batch
        in amazon_reviews_db.query("All_Beauty")
    ]
)
reviews["sentiment"] = reviews["rating"].apply(
    lambda rating: 1 if rating > 3 else 0 if rating < 3 else -1
)
items = pd.concat(
    objs = [
        batch
        for batch
        in amazon_items_db.query("All_Beauty")
    ]
)
pd.merge(
    left = reviews,
    right = items,
    on = "parent_asin",
    how = "left"
)